In [10]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [11]:
# random.sample(range(0, len(image_names)), 5)

In [12]:
image_names = get_png_names("../data/MEX/")[0:500]
# image_names = get_png_names("../pooling/data/MEX2/")
# image_indices = random.sample(range(0, len(image_names)), 1000)
# image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../../pooling/data/migration_data.json" , image_names)

500 municipalities.


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

butler = geozoom_handler(attn_model, 
                         device, 
                         criterion, 
                         attn_optimizer, 
                         num_thresholds = 4,
                         reduction_percent = .75,
                         convergence_dims = (224, 224),
                         change_bounds = (-50, 50),
                         perc_change_thresh = .75, 
                         plot = False, 
                         v = False)

In [14]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [15]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [7]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  15
Num validation:  10


In [8]:
# for i,o in train_dl:
#     print(i)
# #     print(load_inputs(i[0]).shape, o)

In [ ]:
butler.train_attn_model(train_dl, val_dl)

/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch:  0
  Training Loss:  2728.3689577738446
  Validation Loss:  4769.813626098633


Epoch:  1
  Training Loss:  2471.105067138672
  Validation Loss:  4601.63484161377


Epoch:  2
  Training Loss:  2353.54746866862
  Validation Loss:  4556.031277160644


Epoch:  3
  Training Loss:  2321.5247540283203
  Validation Loss:  4555.6955490112305


Epoch:  4
  Training Loss:  2311.3223319498697
  Validation Loss:  4576.666403503418
Percentage above threshold:  0.6166666666666667


Epoch:  5
  Training Loss:  2307.3946081542967
  Validation Loss:  4528.04497467041
Percentage above threshold:  0.6366666666666667


Epoch:  6
  Training Loss:  2301.699991455078
  Validation Loss:  4490.824416809082
Percentage above threshold:  0.65


Epoch:  7
  Training Loss:  2292.913971761068
  Validation Loss:  4561.835305786133
Percentage above threshold:  0.6233333333333333


Epoch:  8
  Training Loss:  2301.3270544433594
  Validation Loss:  4705.168775634766
Percentage above threshold:  0.7033333333333334

In [14]:
butler.train_fc_model(train_dl, val_dl, outside_estimator = 'rf')

Random Forest MAE:  1455.3926666666669


In [15]:
image_names

['../data/MEX/484001008/pngs/484001008_2010_all_box484001008_MAY.png',
 '../data/MEX/484004004/pngs/484004004_2010_all_box484004004_MAY.png',
 '../data/MEX/484004001/pngs/484004001_2010_all_box484004001_MAY.png',
 '../data/MEX/484005010/pngs/484005010_2010_all_box484005010_MAY.png',
 '../data/MEX/484001005/pngs/484001005_2010_all_box484001005_MAY.png',
 '../data/MEX/484001007/pngs/484001007_2010_all_box484001007_MAY.png',
 '../data/MEX/484001006/pngs/484001006_2010_all_box484001006_MAY.png',
 '../data/MEX/484001001/pngs/484001001_2010_all_box484001001_MAY.png',
 '../data/MEX/484004005/pngs/484004005_2010_all_box484004005_MAY.png',
 '../data/MEX/484005001/pngs/484005001_2010_all_box484005001_MAY.png',
 '../data/MEX/484005004/pngs/484005004_2010_all_box484005004_MAY.png',
 '../data/MEX/484005008/pngs/484005008_2010_all_box484005008_MAY.png',
 '../data/MEX/484001002/pngs/484001002_2010_all_box484001002_MAY.png',
 '../data/MEX/484004006/pngs/484004006_2010_all_box484004006_MAY.png',
 '../d

In [19]:
butler.plot_attn_map(impath = '../data/MEX/484007070/pngs/484007070_2010_all_box484007070_MAY.png')


<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>